In [1]:
# text classification

# let's make predictions!


# End goal:

# model.predict(my_text)  # it'll tell us the classification of that text

In [2]:
# two basic types of machine learning:
# - supervised learning
#     teaching the computer with examples
#     asking it to make predictions based on examples

#     - classification
#     - regression

# - unsupervised learning

In [3]:
# within machine learning
# within supervised learning
# within classification

# text classification

In [4]:
# classify by source
# classify by meaning
# classify by age

# spam filter

In [ ]:
# (1) get some input text, for which we know the classification
# (2) clean up the text / put it into a format the model could use
# (3) part of that is: defining a vocabulary and creating a term matrix
# (4) create the model
# (5) split our data into training and testing data
# (6) make some predictions
# (7) check -- how good are our predict